# Training a Convolutional Neural Network

In this exercise, you will have to create a CNN model and then train it on the CIFAR10 dataset. The data loading and model training, testing logic are already included in your code. Infact, they are the same as for the Feed Forward Neural Network you built in the last exercises.

Here are the steps you need to do to complete this exercise:

1. In Starter Code below, finish the `Model()` class. These should contain the code that defines the layers of your model in the `__init__()` function and the model execution in the `forward()` function.
2. Add a cost function and optimizer. You can use the same cost functions and optimizer from the previous exercise.
3. Run the cells to make sure that the model is training properly.

In case you get stuck, you can look at the solution by clicking the jupyter symbol at the top left and navigating to `training_a_cnn_solution.ipynb`.

## Try It Out!
- Play around with the number of layers and filters in your model. How does the accuracy change? How long does it take to train the model?
- Try to train your model with some other types of convolutional layers like depthwise separable convolutions
- Can you create the same network in TensorFlow as well?


## Package Installations
**NOTE**: Everytime you start the GPU, run this before your code. 

In [1]:
!pip install ipywidgets
!pip list

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 121 kB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 29.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 37.0 MB/s eta 0:00:01
Package                    Version            
-------------------------- -------------------
absl-py                    0.9.0              
aequitas                   0.38.0             
apache-beam                2.19.0             
asn1crypto                 1.3.0              
astor                      0.7.1              
attrs                      19.3.0             
avro-python3               1.9.1              
backcall                   0.1.0              
bleach                     3.1.1              
blinker                    1.4                
boto3                      1.12.13            
botocore                   1.15.13            
cachetools                 3.1.1              
certifi    

Flask                      0.12.2             
Flask-Bootstrap            3.3.7.1            
future                     0.18.2             
gast                       0.2.2              
google-api-core            1.16.0             
google-apitools            0.5.28             
google-auth                1.11.2             
google-auth-oauthlib       0.4.1              
google-cloud-bigquery      1.17.1             
google-cloud-bigtable      1.0.0              
google-cloud-core          1.3.0              
google-cloud-datastore     1.7.4              
google-cloud-pubsub        1.0.2              
google-pasta               0.1.8              
google-resumable-media     0.4.1              
googleapis-common-protos   1.51.0             
grpc-google-iam-v1         0.12.3             
grpcio                     1.27.2             
h5py                       2.10.0             
hdfs                       2.5.8              
html5lib                   1.0.1              
httplib2     

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Starter Code

**Remember** to DISABLE the GPU when you are not training.

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms

def train(model, train_loader, cost, optimizer, epoch):
    model.train()
    for e in range(epoch):
        running_loss=0
        correct=0
        for data, target in train_loader:
            optimizer.zero_grad()
            #NOTE: Notice how we are not changing the data shape here
            # This is because CNNs expects a 3 dimensional input
            pred = model(data)
            loss = cost(pred, target)
            running_loss+=loss
            loss.backward()
            optimizer.step()
            pred=pred.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
        print(f"Epoch {e}: Loss {running_loss/len(train_loader.dataset)}, Accuracy {100*(correct/len(train_loader.dataset))}%")

def test(model, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            #NOTE: Notice how we are not changing the data shape here
            # This is because CNNs expects a 3 dimensional input
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    print(f'Test set: Accuracy: {correct}/{len(test_loader.dataset)} = {100*(correct/len(test_loader.dataset))}%)')

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        # TODO: Define the layers you need in your model
        self.conv1 = nn.Conv2d(3, 6, 5) # (Channels, # Kernel, Kernel Size)
        self.pool = nn.MaxPool2d(2,2) # (Kernel Size, Stride)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 16, 5)
        self.fcn1 = nn.Linear(20 * 5 * 5, 256)
        self.fcn2 = nn.Linear(256, 128)
        self.fcn3 = nn.Linear(128, 84)
        self.fcn4 = nn.Linear(84, 64)
        self.fcn5 = nn.Linear(64, 10)
        
#         Entrada =  torch.Size([64, 3, 32, 32])
#         Saída Conv1 torch.Size([64, 6, 14, 14])
#         Saída Conv2 torch.Size([64, 16, 5, 5])

    def forward(self, x):
        #TODO: Define your model execution
        print('Entrada = ', x.shape)
        x = self.conv1(x) # Entrada da imagem na primeira layer de convolução
        x = F.relu(x) # Aplicação de ReLU
        x = self.pool(x) # Pooling p/ downsampling e entrada em uma nova camada
        print('Saída Conv1',x.shape)
        # de convolução
        x = self.conv2(x) # Entrada da saída do pooling para nova convolução
        x = F.relu(x) # Aplicação de ReLU na saída da convolução
        x = self.pool(x) # Downsampling da saída da segunda layer de convolução
        print('Saída Conv2',x.shape)
        x = self.conv3(x) # Entrada da saída do pooling para nova convolução
        x = F.relu(x) # Aplicação de ReLU na saída da convolução
        x = self.pool(x) # Downsampling da saída da segunda layer de convolução
        print('Saída Conv3',x.shape)
        
        # --- Final das Layers de Convolução ---
        
        x = torch.flatten(x,1) # Transformação para vetor 1-D de entrada na 
        # Fully Connected Network
        x = self.fcn1(x) # Input inicial na FCN vindo das layers de convolução
        x = F.relu(x) # Aplicação da ReLU
        x = self.fcn2(x) # Hidden Layer
        x = F.relu(x) # Aplicação da ReLU
        x = self.fcn3(x) # Hidden Layer
        x = F.relu(x) # Aplicação da ReLU
        x = self.fcn4(x) # Hidden Layer
        x = F.relu(x) # Aplicação da ReLU
        x = self.fcn5(x)
        return x


batch_size = 32 
epoch = 1

training_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

testing_transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
        download=True, transform=training_transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
        shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
        download=True, transform=testing_transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
        shuffle=False)

model=Model()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train(model, trainloader, criterion, optimizer, epoch)
test(model, testloader)


Files already downloaded and verified
Files already downloaded and verified
Entrada =  torch.Size([32, 3, 32, 32])
Saída Conv1 torch.Size([32, 6, 14, 14])
Saída Conv2 torch.Size([32, 16, 5, 5])


RuntimeError: Given input size: (16x1x1). Calculated output size: (16x0x0). Output size is too small